In [101]:
# Import section
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode

## Test with 1 data set

In [113]:
data1 = pd.read_csv('/Users/thomaskremer/code/black-dalia/event-forecast-rio/raw_data/parte1.csv', sep=';', encoding = 'iso-8859-1')
kept_columns = ['controle', 'titulo_do', 'total_rbft', 'cisp', 'data_fato', 'hora_fato', 'local', 'bairro_fato']
eng_columns = ['Crime_ID', 'Crime_sub_type', 'Crime_type', 'Police_station', 'Date' , 'Time', 'Place_type', 'Neighborhood']

In [114]:
data1 = data1[data1['municipio_fato'] == 'Rio de Janeiro (Capital)'] # Filtering on Rio de Janeiro only
                 
data1 = data1[kept_columns] # Removing useless columns
                 
data1.columns = eng_columns  # Renaming columns in English

data1 = data1.drop_duplicates(subset="Crime_ID") # Removing duplicates

data1 = data1[data1['Time'] != '99'] # Removing invalid time format
data1['Date_Time'] = pd.to_datetime(data1['Date'] + ' ' 
                                  + data1['Time'], format='%Y-%m-%d %H:%M') # Passing to datetime format
data1.drop(columns=['Date', 'Time'], inplace=True) # Removing time and date columns once the Date_Time is created

data1 = data1[data1['Date_Time'] > '2008-01-31'] # Removing irrelevant date samples

data1['Neighborhood'] = data1['Neighborhood'].map(lambda x: unidecode.unidecode(x)) # Removing accents

data1 = data1[data1['Neighborhood'] != 'sem informacao'] # Removing missing values for neighborhood

data1

,Crime_ID,Crime_sub_type,Crime_type,Police_station,Place_type,Neighborhood,Date_Time
0,00787401-2009,Lesão corporal dolosa,Não se aplica,043a. Pedra de Guaratiba,Praia,PRAIA DA BRISA/PEDRA DE GUARAT,2009-01-01 04:30:00
5,01137431-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Residência,SANTISSIMO,2009-01-01 19:00:00
9,02023941-2009,Lesão corporal dolosa,Não se aplica,040a. Honório Gurgel,Residência,HONORIO GURGEL,2008-12-31 20:30:00
15,02646521-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Residência,VASCONCELOS,2009-01-01 19:00:00
19,04751871-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Via pública,CAMPO GRANDE,2009-01-01 01:30:00
...,...,...,...,...,...,...,...
220373,95665521-2009,Lesão corporal culposa de trânsito,Não se aplica,039a. Pavuna,Via pública,ACARI,2009-12-31 02:01:00
220378,96210311-2009,Lesão corporal dolosa,Não se aplica,016a. Barra da Tijuca,Residência,ITANHANGA,2009-12-29 07:00:00
220384,97428811-2009,Lesão corporal culposa de trânsito,Não se aplica,043a. Pedra de Guaratiba,Via pública,PEDRA DE GUARATIBA,2009-12-31 17:00:00
220391,98703441-2009,Ameaça,Não se aplica,043a. Pedra de Guaratiba,Residência,GUARATIBA,2009-12-31 03:00:00


In [99]:
(data1['Neighborhood'] == 'sem informação').sum()

1606

## With all data 

In [115]:
# Data loading

data1 = pd.read_csv('/Users/thomaskremer/code/black-dalia/event-forecast-rio/raw_data/parte1.csv', sep=';', encoding = 'iso-8859-1')
data2 = pd.read_csv('/Users/thomaskremer/code/black-dalia/event-forecast-rio/raw_data/parte2.csv', sep=';', encoding = 'iso-8859-1')
data3 = pd.read_csv('/Users/thomaskremer/code/black-dalia/event-forecast-rio/raw_data/parte3.csv', sep=';', encoding = 'iso-8859-1')
data4 = pd.read_csv('/Users/thomaskremer/code/black-dalia/event-forecast-rio/raw_data/parte4.csv', sep=';', encoding = 'iso-8859-1')
kept_columns = ['controle', 'titulo_do', 'total_rbft', 'cisp', 'data_fato', 'hora_fato', 'local', 'bairro_fato']
eng_columns = ['Crime_ID', 'Crime_sub_type', 'Crime_type', 'Police_station', 'Date', 'Time', 'Place_type', 'Neighborhood']

In [116]:
data = pd.concat([data1, data2, data3, data4]) # Merging all datasets
data = data[data['municipio_fato'] == 'Rio de Janeiro (Capital)'] # Filtering on Rio de Janeiro only
                 
data = data[kept_columns] # Removing useless columns
                 
data.columns = eng_columns  # Renaming columns in English

data = data.drop_duplicates(subset="Crime_ID") # Removing duplicates

data = data[data['Time'] != '99'] # Removing invalid time format
data['Date_Time'] = pd.to_datetime(data['Date'] + ' ' 
                                  + data['Time'], format='%Y-%m-%d %H:%M') # Passing to datetime format
data.drop(columns=['Date', 'Time'], inplace=True) # Removing time and date columns once the Date_Time is created

data = data[data['Date_Time'] > '2008-01-31'] # Removing irrelevant date samples

data['Neighborhood'] = data['Neighborhood'].map(lambda x: unidecode.unidecode(x)) # Removing accents

data = data[data['Neighborhood'] != 'sem informacao'] # Removing missing values for neighborhood

data

,Crime_ID,Crime_sub_type,Crime_type,Police_station,Place_type,Neighborhood,Date_Time
0,00787401-2009,Lesão corporal dolosa,Não se aplica,043a. Pedra de Guaratiba,Praia,PRAIA DA BRISA/PEDRA DE GUARAT,2009-01-01 04:30:00
5,01137431-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Residência,SANTISSIMO,2009-01-01 19:00:00
9,02023941-2009,Lesão corporal dolosa,Não se aplica,040a. Honório Gurgel,Residência,HONORIO GURGEL,2008-12-31 20:30:00
15,02646521-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Residência,VASCONCELOS,2009-01-01 19:00:00
19,04751871-2009,Lesão corporal dolosa,Não se aplica,035a. Campo Grande,Via pública,CAMPO GRANDE,2009-01-01 01:30:00
...,...,...,...,...,...,...,...
701655,96730181-2019,Roubo em coletivo,Roubo,027a. Vicente de Carvalho,Interior de coletivo,Iraja,2019-12-19 20:55:00
701658,97578571-2019,Furto a transeunte,Furto,012a. Copacabana,Praia,Copacabana,2019-12-31 17:00:00
701661,98313421-2019,Furto a transeunte,Furto,012a. Copacabana,Via pública,Copacabana,2019-12-29 19:00:00
701662,98452031-2019,Furto a transeunte,Furto,014a. Leblon,Interior de coletivo,Centro,2019-12-30 11:45:00
